In [1]:
#imports
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
import sys
#sys.path.append("../BerkeleySETI/Messy-Notebooks")
sys.path.append("../FinalClassifier/data")
import os

import data
from data import loaders
from data import phasefold as pf
import lightkurve as lk
import warnings
import glob
import pickle

%load_ext autoreload
%autoreload 2

In [2]:
#set up
warnings.filterwarnings("ignore", category=RuntimeWarning) #fset = pd.read_csv('sect6data.csv')
data_dir = "/mnt/disks/lcs/"
data.mount_drive(data_dir)
data_dir = "/mnt/disks/lcs/tess-goddard-lcs/"

Disk mounted


___
We want to get the features that we will use for our classifier
<br><b> *Note: Be sure we have the correct 62 features, with mstat and astat

In [3]:
#import sector 6 features
!gcsfuse --implicit-dirs tess-goddard-lcs /home/jupyter/mountpoint
datafile = "/home/jupyter/mountpoint/dataproducts/db.v2.h5"
scores6 = pd.read_hdf(datafile, f'S6/scores')
features6 = pd.read_hdf(datafile, f'S6/features')

2023/01/18 18:02:36.362994 Start gcsfuse/0.39.2 (Go version go1.17.5) for app "" using mount point: /home/jupyter/mountpoint
2023/01/18 18:02:36.375573 Opening GCS connection...
2023/01/18 18:02:36.539898 Mounting file system "tess-goddard-lcs"...
2023/01/18 18:02:36.636982 File system has been successfully mounted.


In [7]:
features6

____
You don't need to run the below section, as feats.csv has already been provided as training data. This is just for your own reference if you're wondering where it came from

In [8]:
#list of ebs that we will use for our training data
txt_file = open("data/ebs_s6.txt", "r")
file_content = txt_file.read()
content_list = file_content.split("\n")
txt_file.close()
eb = [eval(i) for i in content_list]

#list of eb tics
eb

[235231072,
 301185875,
 301185877,
 234169663,
 89016404,
 80836949,
 140336965,
 156501953,
 282503011,
 715766691,
 301185891,
 436619704,
 39354700,
 220140809,
 156241056,
 285000038,
 285000042,
 234065348,
 265899106,
 39838127,
 234878308,
 43144900,
 206794153,
 265715239,
 207004915,
 235548197,
 247681431,
 36288537,
 119461266,
 234169642,
 237590010,
 39667795,
 235552142,
 79635172,
 369673753,
 280212098,
 266193357,
 143215473,
 234926584,
 445955531,
 712487688,
 279900791,
 387690877,
 42991740,
 438545375,
 92009366,
 159086692,
 247556517,
 284199394,
 231161386,
 445955532,
 445955536,
 206840164,
 206840192,
 234158088,
 308387164,
 60242904,
 714514062,
 139476794,
 139476806,
 308169172,
 159057212,
 309908733,
 349480507,
 42885762,
 206840156,
 206850953,
 234146623,
 308169197,
 60242900,
 220149256,
 231959604,
 42991745,
 140209204,
 237647557,
 264594192,
 264594193,
 264594194,
 281157204,
 138904948,
 280304732,
 437918915,
 97568437,
 247681430,
 369681

In [9]:
f6eb = features6.loc[eb]

In [10]:
#f6eb - corresponding 62 features for each eb
f6eb

,longtermtrend,meanmedrat,skews,varss,coeffvar,stds,numoutliers,numnegoutliers,numposoutliers,numout1s,...,sautocorrcoef,autocorrcoef,flatmean,tflatmean,roundmean,troundmean,roundrat,flatrat,mstat,astat
235231072,0.026648,0.991050,0.000710,0.026889,94.0,0.0,0.0,0.0,0.000851,0.024911,...,0.881838,0.009123,0.013710,0.018850,0.026099,0.428787,0.108102,0.042574,1.569,0.000000
301185875,0.081945,0.971039,0.006715,0.084389,97.0,0.0,6.0,6.0,0.001877,0.500361,...,0.864516,0.004411,0.009037,0.013953,0.030553,0.436112,0.363679,0.042226,1.561,0.838213
301185877,0.081945,0.971039,0.006715,0.084389,97.0,0.0,6.0,6.0,0.001877,0.500361,...,0.864516,0.004411,0.009037,0.013953,0.030553,0.436112,0.363679,0.042226,1.561,0.838212
234169663,0.044620,0.984330,0.001991,0.045330,95.0,0.0,1.0,1.0,0.001174,0.165540,...,0.864516,0.006410,0.011026,0.016200,0.026155,0.478317,0.206123,0.024256,1.559,0.673318
89016404,0.086715,0.966162,0.007520,0.089752,112.0,0.0,0.0,0.0,0.002893,0.599635,...,0.835498,0.007718,0.014056,0.023216,0.110242,0.612072,0.357053,0.079822,1.558,0.848091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271643533,0.159520,0.949524,0.025447,0.168000,387.0,0.0,1.0,1.0,0.110968,5.774229,...,0.176963,0.249553,0.441912,0.638240,0.819257,0.941121,0.968383,0.385284,0.623,0.934289
436423408,0.214966,0.928026,0.046210,0.231638,361.0,0.0,0.0,0.0,0.147812,5.761445,...,0.211039,0.227504,0.422346,0.601965,0.813311,0.927010,0.507398,0.340901,0.622,0.901662
33865219,0.178479,0.943878,0.031855,0.189092,339.0,0.0,1.0,1.0,0.118789,6.655849,...,0.214831,0.226883,0.412879,0.600523,0.794949,0.928034,1.000000,0.410119,0.621,0.899500
290039579,0.184797,0.941332,0.034150,0.196315,374.0,0.0,1.0,1.0,0.128155,5.482608,...,0.186848,0.235463,0.431017,0.645656,0.826721,0.941337,0.910452,0.456640,0.596,0.926016


In [11]:
#list of ebs from villanova (should be exhaustive in tmcl), which we will use to find our noneb dataset
tmclebs = pd.read_csv('data/allsectorstmclebs.csv')

ref = pd.read_csv(data_dir+f"sector6lookup.csv")
tmcl = ["2_min_cadence" in fn for fn in ref.Filename]
reftmcl = ref[tmcl]
z = ~reftmcl.TIC_ID.isin(tmclebs.TIC_ID.values)
f6nonebs = reftmcl[z].sample(frac=1)

In [12]:
#features for our noneb
f6noneb = features6.loc[f6nonebs.TIC_ID.values]
f6noneb

,longtermtrend,meanmedrat,skews,varss,coeffvar,stds,numoutliers,numnegoutliers,numposoutliers,numout1s,...,sautocorrcoef,autocorrcoef,flatmean,tflatmean,roundmean,troundmean,roundrat,flatrat,mstat,astat
219391354,0.033081,0.998880,0.001094,0.033118,1.0,0.0,1.0,1.0,0.000399,0.014936,...,0.352683,0.153085,0.282498,0.427426,0.591036,0.781410,1.000000,0.642724,0.166,0.157662
237945948,0.032274,0.999106,0.001042,0.032303,1.0,0.0,1.0,1.0,0.001524,0.031268,...,0.457573,0.137575,0.266601,0.384374,0.493088,0.681576,0.981914,1.232854,0.133,0.200462
144542681,0.032326,0.998942,0.001045,0.032361,1.0,0.0,1.0,1.0,0.000138,0.008094,...,0.331590,0.171410,0.311590,0.435061,0.576879,0.802188,1.000000,0.728844,0.169,0.054424
340889146,0.033670,0.998298,0.001134,0.033728,7.0,0.0,1.0,1.0,0.005083,0.297421,...,0.511530,0.121852,0.232384,0.340835,0.483029,0.700549,1.000000,0.677215,0.217,0.119149
219410336,0.033077,0.998908,0.001094,0.033114,1.0,0.0,1.0,1.0,0.000065,0.003708,...,0.681271,0.128145,0.235554,0.352913,0.485649,0.724491,1.000000,0.714768,0.165,0.143790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55499665,0.033008,0.998938,0.001090,0.033043,1.0,0.0,1.0,1.0,0.000192,0.015939,...,0.484188,0.153132,0.259040,0.376154,0.537394,0.784141,1.000000,2.154652,0.166,0.157039
260609222,0.032479,0.998942,0.001055,0.032513,1.0,0.0,1.0,1.0,0.000105,0.009529,...,0.381204,0.140050,0.261325,0.381428,0.547884,0.761928,1.000000,0.700787,0.162,0.105266
219184046,0.041700,1.000234,0.001739,0.041691,211.0,1.0,1.0,2.0,0.021358,1.700960,...,0.436090,0.132596,0.231959,0.355573,0.519579,0.743861,0.645649,3.547824,0.028,0.220067
41030824,0.032344,0.998950,0.001046,0.032378,1.0,0.0,1.0,1.0,0.000132,0.010840,...,0.452356,0.155225,0.247759,0.363138,0.541412,0.777182,1.000000,1.538343,0.159,0.083705


In [10]:
#new dataset (3) = f6eb + f6noneb
frames = [f6noneb, f6eb]
feats = pd.concat(frames)
feats.to_csv('data/feats.csv')

___
ML portion here

In [13]:
#run this line to read in feats.csv
feats = pd.read_csv('data/feats.csv', index_col = 0)

In [14]:
train, test = train_test_split(np.arange(len(feats)), train_size=0.8)
labels = []

for i in range(len(f6noneb)):
    labels.append('NonEB')
for i in range(len(f6noneb),len(f6eb)+len(f6noneb)):
    labels.append('EB')
    
X_trainfull = feats.iloc[train]
X_testfull = feats.iloc[test]

y_train = []
y_test = []
for i in train:
    y_train.append(labels[i])
for i in test:
    y_test.append(labels[i])
    
y_train = pd.Series(data = y_train, index = train)
y_test = pd.Series(data = y_test, index = test)

In [15]:
newy_train = []
for i in (y_train.index.values):
    if (y_train[i]=='NonEB'):
        newy_train.append(0)
    else:
        newy_train.append(1)
y_train = pd.Series(data = newy_train, index = train)

newy_test = []
for i in (y_test.index.values):
    if (y_test[i]=='NonEB'):
        newy_test.append(0)
    else:
        newy_test.append(1)
y_test = pd.Series(data = newy_test, index = test)

In [16]:
X_train = X_trainfull
X_test = X_testfull

In [16]:
X_train

,longtermtrend,meanmedrat,skews,varss,coeffvar,stds,numoutliers,numnegoutliers,numposoutliers,numout1s,...,sautocorrcoef,autocorrcoef,flatmean,tflatmean,roundmean,troundmean,roundrat,flatrat,mstat,astat
32034265,0.033009,0.998936,0.001090,0.033044,1.0,0.0,1.0,1.0,0.000280,0.015626,...,0.432933,0.143311,0.264984,0.393812,0.532299,0.766463,1.000000,1.273311,0.164,0.235399
238949526,0.032758,0.998940,0.001073,0.032793,1.0,0.0,1.0,1.0,0.000201,0.012172,...,0.371643,0.150027,0.270864,0.412247,0.558991,0.769680,1.000000,0.641473,0.162,0.232014
67197292,0.032294,0.998965,0.001043,0.032328,1.0,0.0,1.0,1.0,0.000257,0.021495,...,0.376827,0.147975,0.266305,0.398647,0.565261,0.770930,1.000000,0.814851,0.160,0.076123
255683786,0.032777,0.998858,0.001074,0.032814,1.0,0.0,1.0,1.0,0.000834,0.012149,...,0.335124,0.154607,0.302515,0.408824,0.549936,0.751167,1.000000,0.571830,0.165,0.631518
24948729,0.032404,0.998913,0.001050,0.032439,1.0,0.0,1.0,1.0,0.001013,0.086636,...,0.402516,0.132812,0.246446,0.360943,0.520940,0.751524,1.000000,0.663836,0.167,0.031053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278899461,0.032479,0.998945,0.001055,0.032514,1.0,0.0,1.0,1.0,0.000153,0.007374,...,0.366420,0.155859,0.269150,0.423038,0.582543,0.804810,1.000000,0.724643,0.161,0.406353
167125688,0.032348,0.998960,0.001046,0.032382,1.0,0.0,1.0,1.0,0.000335,0.025409,...,0.518325,0.141360,0.249840,0.374989,0.520015,0.765963,1.000000,0.806458,0.164,0.173805
219183103,0.032757,0.998931,0.001073,0.032792,1.0,0.0,1.0,1.0,0.000181,0.012155,...,0.392052,0.141220,0.259406,0.385886,0.538042,0.743628,1.000000,0.591531,0.163,0.099505
251236566,0.089483,0.959907,0.008007,0.093220,165.0,0.0,0.0,0.0,0.015844,0.653571,...,0.693548,0.040739,0.083116,0.206994,0.511864,0.830851,0.296436,0.169804,1.249,0.867780


In [17]:
RF = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators = 500, max_depth = 15,random_state=31415))
RF.fit(X_train,y_train)
y_predRF = RF.predict(X_test)
print(accuracy_score(y_test,y_predRF)) # Fraction of classifications that were correct
print(precision_score(y_test,y_predRF,pos_label=1))

0.999274135010888
0.9961464354527938


In [18]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_predRF)

array([[3613,    2],
       [   1,  517]])

In [19]:
featswithlabels = feats
featswithlabels['Label'] = labels
featswithlabels = featswithlabels.sample(frac=1,random_state = 31415)
ylabel = featswithlabels['Label']
y = pd.Series(ylabel.values)
featswithlabels = featswithlabels.drop(columns=['Label'])
X = featswithlabels

In [20]:
models = [RF]
for i in models:
    scores = cross_val_score(i, X, y, cv=10)
    print(f'{i.named_steps}: {sum(scores)/10}')

{'standardscaler': StandardScaler(), 'randomforestclassifier': RandomForestClassifier(max_depth=15, n_estimators=500, random_state=31415)}: 0.9995160431451506


__________
This assumes that astats is already in the datafile, and can be imported along with the other features

In [4]:
#sector you want to classify
sector = 1

In [22]:
#create the directory for each sector
os.mkdir(f'Sector{sector}')

FileExistsError: [Errno 17] File exists: 'Sector1'

In [5]:
#change the variable toClassify to data that you want to classify
toClassify = pd.read_hdf(datafile, f'S{sector}/features')
refsect = pd.read_csv(data_dir+f"sector{sector}lookup.csv")

In [26]:
y_pred = RF.predict(toClassify)
y_predprob = RF.predict_proba(toClassify)

In [27]:
y_probzero = []
y_probone = []
for i in range(len(y_predprob)):
    y_probzero.append(y_predprob[i][0])
    y_probone.append(y_predprob[i][1])

In [47]:
predictions = pd.DataFrame(index = toClassify.index.values)
predictions['probzero'] = y_probzero
predictions['probone'] = y_probone
predictions['pred'] = y_pred
predictions = predictions.sort_values(by=['probone'], ascending = False)

In [48]:
predictions

,probzero,probone,pred
140608211,0.0,1.0,1
231791552,0.0,1.0,1
300740040,0.0,1.0,1
181052118,0.0,1.0,1
38811805,0.0,1.0,1
...,...,...,...
389505262,1.0,0.0,0
280048495,1.0,0.0,0
404808101,1.0,0.0,0
284196526,1.0,0.0,0


In [50]:
rslt_df = predictions[predictions['pred'] == 1]
rslt_df.to_csv(f'Sector{sector}/predSector{sector}.csv')
predictions.to_csv(f'Sector{sector}/sect{sector}predandprob.csv')

In [59]:
p = predictions[predictions['probone'] == 1]

,probzero,probone,pred
140608211,0.0,1.0,1
231791552,0.0,1.0,1
300740040,0.0,1.0,1
181052118,0.0,1.0,1
38811805,0.0,1.0,1
...,...,...,...
260950407,0.0,1.0,1
349641813,0.0,1.0,1
33734375,0.0,1.0,1
141122198,0.0,1.0,1


In [68]:
p = predictions[predictions['probone'] > 0.9]
p

,probzero,probone,pred
140608211,0.000,1.000,1
231791552,0.000,1.000,1
300740040,0.000,1.000,1
181052118,0.000,1.000,1
38811805,0.000,1.000,1
...,...,...,...
206498836,0.098,0.902,1
349683770,0.098,0.902,1
419588384,0.098,0.902,1
281630204,0.098,0.902,1


In [69]:
p = predictions[predictions['probone'] > 0.5]
p = p[p['probone'] < 0.9]
p

,probzero,probone,pred
270221859,0.102000,0.898000,1
126803454,0.102000,0.898000,1
206488769,0.102000,0.898000,1
231702259,0.102000,0.898000,1
1989908229,0.102000,0.898000,1
...,...,...,...
425064789,0.498000,0.502000,1
212255303,0.498000,0.502000,1
269883697,0.499000,0.501000,1
55274131,0.499000,0.501000,1


In [51]:
#rslt_df = pd.read_csv(f'predSector{sector}.csv', index_col = 0)
values = rslt_df.index.values
values = values.tolist()
with open(f"Sector{sector}/ticpredictionsSector{sector}.txt", "w") as output:
    for i in range(len(values)):
        output.write(str(values[i]))
        output.write("\n")

In [52]:
values

[140608211,
 231791552,
 300740040,
 181052118,
 38811805,
 303909865,
 126605150,
 270203709,
 204497617,
 307364443,
 117544914,
 89018905,
 349835374,
 316852879,
 33715938,
 316937318,
 126945917,
 306573146,
 370239669,
 126944778,
 273550448,
 39017668,
 220388567,
 290495367,
 327939046,
 177236393,
 2026930048,
 306665786,
 31798977,
 238891804,
 232087348,
 231093356,
 278666918,
 219170666,
 261265716,
 33734376,
 267091837,
 141866705,
 89043105,
 92259410,
 425935688,
 294671852,
 220491219,
 389021724,
 117544922,
 306900745,
 410354938,
 302751961,
 300014528,
 270174474,
 201256776,
 181886413,
 355545390,
 234524824,
 231714659,
 234285429,
 219335483,
 177164595,
 389020350,
 389020349,
 30266257,
 260503860,
 309781192,
 734588359,
 300013254,
 167750920,
 314887863,
 141268467,
 300657354,
 219159872,
 277773589,
 167651110,
 355446601,
 382627633,
 126945904,
 126945919,
 150389733,
 33862174,
 150390197,
 394700186,
 150361911,
 260541900,
 382627629,
 140761918,
 

---

In [65]:
sec1 = pd.read_csv('Sector1/sect1predandprob.csv', index_col=0)

In [66]:
eblist = sec1[sec1['probone'] > 0.5]
eblist

,probzero,probone,pred
140608211,0.000000,1.000000,1
231791552,0.000000,1.000000,1
300740040,0.000000,1.000000,1
181052118,0.000000,1.000000,1
38811805,0.000000,1.000000,1
...,...,...,...
425064789,0.498000,0.502000,1
212255303,0.498000,0.502000,1
269883697,0.499000,0.501000,1
55274131,0.499000,0.501000,1


In [67]:
values = sec1.index.values
ref = pd.read_csv(data_dir+f"sector1lookup.csv")

In [18]:
values

array([140608211, 231791552, 300740040, ..., 404808101, 284196526,
       255567371])

In [61]:
ebvals = eblist[eblist['probone'] == 1].index.values
ebvals

array([ 140608211,  231791552,  300740040,  181052118,   38811805,
        303909865,  126605150,  270203709,  204497617,  307364443,
        117544914,   89018905,  349835374,  316852879,   33715938,
        316937318,  126945917,  306573146,  370239669,  126944778,
        273550448,   39017668,  220388567,  290495367,  327939046,
        177236393, 2026930048,  306665786,   31798977,  238891804,
        232087348,  231093356,  278666918,  219170666,  261265716,
         33734376,  267091837,  141866705,   89043105,   92259410,
        425935688,  294671852,  220491219,  389021724,  117544922,
        306900745,  410354938,  302751961,  300014528,  270174474,
        201256776,  181886413,  355545390,  234524824,  231714659,
        234285429,  219335483,  177164595,  389020350,  389020349,
         30266257,  260503860,  309781192,  734588359,  300013254,
        167750920,  314887863,  141268467,  300657354,  219159872,
        277773589,  167651110,  355446601,  382627633,  126945

In [62]:
refebs = ref.loc[ref['TIC_ID'].isin(ebvals)]
files = data_dir + refebs.Filename.values
filepath = files[0]

In [63]:
#100%ebs
lcc = loaders.LightCurveCollection(refebs.Sector.values,
                                   refebs.Camera.values,
                                   files,
                                   False
                            )
lcc.useCpus = 8
lcc.load_all_lcs()

lcs = lcc.lcs

In [64]:
import matplotlib.pyplot as plt

from matplotlib.backends.backend_pdf import PdfPages

with PdfPages(f"Sector1/100%ebs.pdf") as pdf:
    plt.cla()
    for i in range(len(ebvals)):
        lcs[i].scatter()
        pdf.savefig()
        plt.close();
plt.close()

In [79]:
#90%-100%ebs
ebvals = eblist[eblist['probone'] > 0.9]
ebvals = ebvals[ebvals['probone'] < 1].index.values
refebs = ref.loc[ref['TIC_ID'].isin(ebvals)]
files = data_dir + refebs.Filename.values
filepath = files[0]
len(ebvals)

1152

In [ ]:
lcc = loaders.LightCurveCollection(refebs.Sector.values,
                                   refebs.Camera.values,
                                   files,
                                   False
                            )
lcc.useCpus = 8
lcc.load_all_lcs()

lcs = lcc.lcs

In [80]:
import matplotlib.pyplot as plt

from matplotlib.backends.backend_pdf import PdfPages

with PdfPages(f"Sector1/90-100%ebs.pdf") as pdf:
    plt.cla()
    for i in range(len(ebvals)):
        lcs[i].scatter()
        pdf.savefig()
        plt.close();
plt.close()

___
Find equal-depth eclipsing binary systems

In [23]:
#rslt_df = pd.read_csv(f'predSector{sector}.csv', index_col = 0)
from data import autopeak as ap

In [24]:
ref = pd.read_csv(data_dir+"sector1lookup.csv")
ref.head()
sec = 1
ref = pd.read_csv(data_dir+f"sector{sec}lookup.csv")
tmcl = ["2_min_cadence" in fn for fn in ref.Filename]

In [25]:
import numpy as np
import matplotlib.pyplot as plt # plotting package
import matplotlib.cm as cm # colormaps
import scipy.signal
import scipy.ndimage.filters
import os
import sys
import fourier
from scipy.optimize import curve_fit
from astropy.io import ascii
from astropy.table import Table
import glob
import re
import pandas as pd

def fnoise(x, a, b, c):
 return a/(x+b)+c

def clean(sequence,n,p):
#Task to iteratively clean a sequence of outliers at the n-sigma level, smoothing every p points.

  smoothseq = scipy.signal.medfilt(sequence, p)
  resid = sequence-smoothseq

# Do iterative sigma clipping of points beyond n-sigma:
  oldstd = scipy.stats.nanstd(resid)
  good = np.where(abs(resid) < n*scipy.stats.nanstd(resid))
  good = good[0]
  newstd = scipy.stats.nanstd(resid[good])
  while (1.-newstd/oldstd > 0.02):
    good1 = np.where(abs(resid[good]) < n*newstd)
    good = good[good1[0]]
    oldstd = newstd
    newstd = scipy.stats.nanstd(resid[good])

  return good

#peak finding algorithm - in autocorrelation
def count(times,values,minamp):
  dtimes=[times[0]]
  zpoints=[values[0]]

  first=(np.where(np.abs(values-values[0]) > minamp))[0]

  if (len(first) != 0):
     dtimes = np.append(dtimes,times[first[0]])
     zpoints = np.append(zpoints,values[first[0]])

     flip=int(np.abs(zpoints[1]-zpoints[0])/(zpoints[1]-zpoints[0]))

     y0=zpoints[0]
     y1=zpoints[1]

     for i in np.arange(first[0]+1,len(times)):
        if (flip == 1):
         if ((values[i]-y0) > (y1-y0)):
           zpoints[len(zpoints)-1]=values[i]
           dtimes[len(zpoints)-1]=times[i]
           y1=values[i]

         if ((y1-values[i]) > minamp):
           zpoints = np.append(zpoints,values[i])
           dtimes = np.append(dtimes,times[i])
           y0=y1
           y1=values[i]
           flip=-1

        if (flip == -1):
         if ((y0-values[i]) > (y0-y1)):
           zpoints[len(zpoints)-1]=values[i]
           dtimes[len(zpoints)-1]=times[i]
           y1=values[i]

         if ((values[i]-y1) > minamp):
           zpoints=np.append(zpoints,values[i])
           dtimes=np.append(dtimes,times[i])
           y0=y1
           y1 = values[i]
           flip = 1

  result = np.array([dtimes,zpoints])

  return result

#use ticid for star
def autopeak(time,flux2,star,sector):

# remove 7-sigma outliers
  time = time[flux2>0]
  flux2 = flux2[flux2>0]
  avg = np.nanmedian(-2.5*np.log10(flux2))
  rms = np.nanstd(-2.5*np.log10(flux2))

  fine = np.where((abs(-2.5*np.log10(flux2)-avg) < 7.*rms) & (~np.isnan(flux2)))
  fine = fine[0]

  avg = np.median(-2.5*np.log10(flux2[fine]))
  rms = np.nanstd(-2.5*np.log10(flux2[fine]))

  mag = -2.5*np.log10(flux2[fine])

  #FIGURE THIS PART OUT  
  if fine.size==0:
    result = np.array([0,1,0,0,1,0])
    return result

# Interpolate onto evenly spaced grid...
  ndatapoints=7040.
  ndays=(time[fine])[len(time[fine])-1]-time[fine[0]]
  u=ndatapoints/ndays

  inttime=(np.arange(ndatapoints,dtype='float')/u)+(time[fine])[0]-(time[fine])[0]
  maginterp=np.interp(inttime,time[fine]-(time[fine])[0],mag)

#  Compute autocorrelation function:
  lag = np.arange(len(maginterp),dtype='float')
  unbias = maginterp-np.mean(maginterp)
  norm = np.sum(unbias**2)    

  ac = np.correlate(unbias,unbias, "full")/norm
  ac = ac[round(len(ac)/2)-1:]

  result=count(lag/u,ac,0.05)
  autopeaks = (result[1])[np.arange(round((len(result[0])+1)/2)-1)*2]
  autopeaktimes = (result[0])[np.arange(round((len(result[0])+1)/2)-1)*2]

  autopeaks = autopeaks[1:len(autopeaks)]
  autopeaktimes = autopeaktimes[1:len(autopeaktimes)]
  
  if (len(autopeaks)>=1):
    bigpeak=np.max(autopeaks)
    bigpeaklag = np.argmax(autopeaks)
    finalper = autopeaktimes[bigpeaklag]
  else:
    finalper = 0.00000000001

  #potential peaks in the autocorrelation functino (plots red dots if exist)
  periodicity = 0
  if (len(autopeaks) >= 1):
   acfit,accov = curve_fit(fnoise,lag/u,ac)
   acsmoothed = acfit[0]/(lag/u+acfit[1])+acfit[2]
   x= np.where(ac == bigpeak)
   x= x[0][0].item()
   peaksmooth = acsmoothed[x]
   periodicity = bigpeak - peaksmooth

   if (len(autopeaks) > 1):
        
  #use bigpeak, then fit slowly varying curve to the autocorrelation (ac)
     if (bigpeak > 1.5*np.max(autopeaks[0:2])):
       finper = finalper
       
     else:
       
       finper = 0
  
  try:  
    if (autopeaks[0]>autopeaks[1]):
        result = 1
    elif (autopeaks[0]<autopeaks[1]):
        result = 2
  except:
    result = 0
  return result

In [26]:
def agraph(tic, remove_outliers = 0):
    row = ref[ref.TIC_ID.isin([tic])]
    string = str(row.Sector)
    s = string.split(" ")
    files = data_dir + ref.Filename.values
    filepath = files[int(s[0])]
    lightc = loaders.load_lc(filepath)
    tic = ref.TIC_ID.values
    tic = tic[int(s[0])]
    sec = ref.Sector.values
    sec = sec[int(s[0])]
    a = autopeak(lightc.time.value,lightc.flux.value,tic,sec)
    #print(a)
    return a

def agraph2(tic, remove_outliers = 0):
    row = ref[ref.TIC_ID.isin([tic])]
    string = str(row.Sector)
    s = string.split(" ")
    files = data_dir + ref.Filename.values
    filepath = files[int(s[0])]
    lightc = loaders.load_lc(filepath)
    tic = ref.TIC_ID.values
    tic = tic[int(s[0])]
    sec = ref.Sector.values
    sec = sec[int(s[0])]
    a = autopeak2(lightc.time.value,lightc.flux.value,tic,sec)
    #print(a)
    return a

In [32]:
%%time
#repeat the below sections for as many lcs there are
classification1000 = []
for i in range(1000):
    a = agraph(rslt_df.index.values[i])
    classification1000.append(a)

CPU times: user 1min 46s, sys: 57.3 s, total: 2min 43s
Wall time: 2min 43s


In [33]:
%%time
classification2000 = []
for i in range(1000,2000):
    a = agraph(rslt_df.index.values[i])
    classification2000.append(a)

CPU times: user 1min 45s, sys: 56.2 s, total: 2min 42s
Wall time: 3min 21s


In [37]:
%%time
classification3000 = []
for i in range(2000,3000):
    a = agraph(rslt_df.index.values[i])
    classification3000.append(a)

CPU times: user 1min 42s, sys: 51 s, total: 2min 33s
Wall time: 2min 33s


In [35]:
%%time
classification4000 = []
for i in range(3000,4000):
    a = agraph(rslt_df.index.values[i])
    classification4000.append(a)

CPU times: user 2min 51s, sys: 1min 14s, total: 4min 6s
Wall time: 9min 24s


In [39]:
%%time
classification5000 = []
for i in range(4000,5000):
    a = agraph(rslt_df.index.values[i])
    classification5000.append(a)

CPU times: user 1min 42s, sys: 49.2 s, total: 2min 31s
Wall time: 3min 3s


In [41]:
%%time
classification6000 = []
for i in range(5000,6000):
    a = agraph(rslt_df.index.values[i])
    classification6000.append(a)

IndexError: index 5255 is out of bounds for axis 0 with size 5255

[2,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 0,
 0,
 2,
 1,
 0,
 2,
 2,
 0,
 2,
 0,
 1,
 array([0, 1, 0, 0, 1, 0]),
 1,
 array([0, 1, 0, 0, 1, 0]),
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 0,
 1,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 2,
 1,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 0,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,


In [42]:
#add them all into one list
#classification = classification500 + classification1000
classification = classification1000 + classification2000 + classification3000 + classification4000 + classification5000 + classification6000

In [94]:
#saves the prediction (1 or 2 or 0) in a txt file
with open(f"Sector{sector}/classificationSector{sector}.txt", "w") as output:
        output.write(str(classification))

In [45]:
d = {'TIC_ID' : rslt_df.index.values , 'Prediction': classification}
prediction = pd.DataFrame(data = d)

In [95]:
prediction.to_csv(f"Sector{sector}/classificationSect{sector}.csv", index=False)
index = prediction[prediction.Prediction.isin([1])]
index

,TIC_ID,Prediction
1,263074592,1
3,261656371,1
6,139721086,1
8,214198381,1
14,441406061,1
...,...,...
5243,149160359,1
5245,370239669,1
5247,2026963422,1
5248,734511288,1


In [48]:
#tic id of "same dip ebs" with estimated around 50% accuracy
tics = []
for i in index.index.values:
    tics.append(rslt_df.index.values[i])

In [97]:
with open(f"Sector{sector}/samedipebsSect{sector}.txt", "w") as output:
    for i in range(len(tics)):
        output.write(str(tics[i]))
        output.write("\n")

________
Phasing and saving the samedipebs

In [53]:
os.mkdir(f'TESSpklFilesSect{sector}')

In [53]:
sec = 1

In [55]:
def calcfold(tic,sec):
    row = ref[ref.TIC_ID.isin([tic])]
    string = str(row.Sector)
    s = string.split(" ")
    files = data_dir + ref.Filename.values
    filepath = files[int(s[0])]
    lightc = loaders.load_lc(filepath)
    period, phased = pf.calcfold(lightc,sec)
    obj = []
    obj.append(tic)
    obj.append(sec)
    obj.append(period.value)
    obj.append(phased.time.value)
    obj.append(phased.flux.value)
    obj.append(phased.flux_err.value)
    obj.append(phased.time_original.value)
    obj.append(phased.quality.value)
    output = open(f'TESSpklFilesSect{sec}/tesslc_{tic}folded.pkl', 'wb')
    pickle.dump(obj, output)
    output.close()

In [51]:
with open(f'Sector{sector}/samedipebsSect{sector}.txt') as f:
    tics = []
    for readline in f: 
        line_strip = readline.strip()
        tics.append(int(line_strip))

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%%time
for i in range(0,250):
    calcfold(tics[i],sec)

In [57]:
%%time
for i in range(250,500):
    calcfold(tics[i],sec)

CPU times: user 13min 21s, sys: 18.4 s, total: 13min 39s
Wall time: 13min 39s


In [72]:
#there seems to be some light kurves that are definitely not ebs, as they are centered around zero (for ex. check out TIC309853236(tics[565]) and TIC212259834(tics[711]))
#we can manually remove this later through visual parsing

In [59]:
%%time
for i in range(500,750):
    calcfold(tics[i],sec)
    print(i)

500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564


/opt/conda/lib/python3.7/site-packages/lightkurve/lightcurve.py:1048: LightkurveWarning: The light curve appears to be zero-centered (median=1.01e+02 +/- 7.73e+02); `normalize()` will divide the light curve by a value close to zero, which is probably not what you want.
  LightkurveWarning,
/opt/conda/lib/python3.7/site-packages/lightkurve/lightcurve.py:1048: LightkurveWarning: The light curve appears to be zero-centered (median=1.01e+02 +/- 7.73e+02); `normalize()` will divide the light curve by a value close to zero, which is probably not what you want.
  LightkurveWarning,


565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710


/opt/conda/lib/python3.7/site-packages/lightkurve/lightcurve.py:1048: LightkurveWarning: The light curve appears to be zero-centered (median=4.25e+01 +/- 1.90e+02); `normalize()` will divide the light curve by a value close to zero, which is probably not what you want.
  LightkurveWarning,
/opt/conda/lib/python3.7/site-packages/lightkurve/lightcurve.py:1048: LightkurveWarning: The light curve appears to be zero-centered (median=4.25e+01 +/- 1.90e+02); `normalize()` will divide the light curve by a value close to zero, which is probably not what you want.
  LightkurveWarning,


711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
CPU times: user 13min 31s, sys: 17.2 s, total: 13min 48s
Wall time: 13min 48s


In [83]:
len(tics)

1935

In [75]:
%%time
for i in range(750,1000):
    calcfold(tics[i],sec)

CPU times: user 13min 59s, sys: 17.7 s, total: 14min 17s
Wall time: 14min 17s


In [77]:
%%time
for i in range(1000,1250):
    calcfold(tics[i],sec)

CPU times: user 14min 13s, sys: 17.3 s, total: 14min 30s
Wall time: 14min 31s


In [78]:
%%time
for i in range(1250,1500):
    calcfold(tics[i],sec)

CPU times: user 14min 6s, sys: 17 s, total: 14min 23s
Wall time: 14min 23s


In [81]:
%%time
for i in range(1500,1750):
    calcfold(tics[i],sec)

CPU times: user 13min 44s, sys: 17.3 s, total: 14min 1s
Wall time: 14min 1s


In [82]:
%%time
for i in range(1750,2000):
    calcfold(tics[i],sec)

IndexError: list index out of range

____
Finding and removing closest stars, then printing the remaining light curves into a pdf file to visually examine

In [99]:
with open(f'Sector{sector}/ticpredictionsSector{sector}.txt') as f:
    tics = []
    for readline in f: 
        line_strip = readline.strip()
        tics.append(int(line_strip))

In [100]:
with open(f'Sector{sector}/samedipebsSect{sector}.txt') as f:
    tics2 = []
    for readline in f: 
        line_strip = readline.strip()
        tics2.append(int(line_strip))

In [101]:
nottics2 = []
for element in tics:
    if element not in tics2:
        nottics2.append(element)

In [102]:
stars = ref.TIC_ID.isin(tics)

In [104]:
#ref7335.csv - csv file for all ebs found
ref[stars].to_csv(f"Sector{sector}/refSect{sector}.csv")

In [105]:
closestarslist = []
for i in range(len(tics)):
    rastar = ref[stars].RA.values[i]
    decstar = ref[stars].Dec.values[i]
    dist = np.sqrt(((rastar-ref[stars].RA.values)*(np.cos(decstar)))**2+(decstar-ref[stars].Dec.values)**2)
    closestars = np.where((dist < 0.1) & (dist != 0))
    closestarslist.append(closestars)

In [128]:
closestarslist

[(array([], dtype=int64),),
 (array([ 396, 1728, 2313]),),
 (array([], dtype=int64),),
 (array([807]),),
 (array([1811]),),
 (array([ 406,  812, 1817]),),
 (array([402]),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([ 848, 1870]),),
 (array([], dtype=int64),),
 (array([1883]),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([1933, 1934, 1945]),),
 (array([], dtype=int64),),
 (array([1964]),),
 (array([ 424, 1966, 2003]),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([444]),),
 (array([ 918,  919, 5215]),),
 (array([], dtype=int64),),
 (array([2177]),),
 (array([], dtype=int64),),
 (array([2242]),),
 (array([2264]),),
 (array([], dtype=int64),),
 (array([ 484, 2273]),),
 (array([], dtype=int64),),
 (array([1005]),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([2298]),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 

In [116]:
closestarstics = []
for i in range (len(closestarslist)):
    tics = []
    tics.append(ref[stars].TIC_ID.values[i])
    for j in range(len(closestarslist[i][0])):
        tics.append(ref[stars].TIC_ID.values[closestarslist[i][0][j]])
    closestarstics.append(tics)

In [129]:
closestarstics

[[269762258],
 [263074592, 263074589, 612155681, 263074593],
 [278706358],
 [261656371, 261656369],
 [152825521, 152825523],
 [139699256, 2026930048, 139699260, 139699248],
 [139721086, 139721085],
 [139841210],
 [214198381],
 [144462697],
 [389726435, 389726434, 389726433],
 [160036449],
 [229057817, 229056112],
 [204497617],
 [441406061],
 [152223725],
 [25226885, 25226877, 25226886, 25200625],
 [280052690],
 [358457398, 299835005],
 [141268467, 141268468, 141268471, 141268474],
 [33862174],
 [267263253],
 [79403459, 79403456],
 [79225651, 79225639, 79225655, 1989169801],
 [394585218],
 [99465976, 99465986],
 [201176965],
 [389476160, 389476161],
 [261746657, 261748317],
 [262958558],
 [290547036, 290547040, 290547046],
 [263078276],
 [33715938, 33715940],
 [303909865],
 [181233908],
 [271596290, 271596289],
 [325469795],
 [220474209],
 [278683641],
 [201256771, 201256776],
 [92352621, 92352620],
 [92352620, 92352621],
 [441422220],
 [89398867],
 [147083089],
 [206462116, 2028782097,

In [110]:
#get lcs for ebs in the larger list thats not in smaller(same depth dip) eb list
refebs = ref.loc[ref['TIC_ID'].isin(nottics2)]
files = data_dir + ref.Filename.values
files = data_dir + refebs.Filename.values
filepath = files[0]
lcc = loaders.LightCurveCollection(refebs.Sector.values,
                                   refebs.Camera.values,
                                   files,
                                   False
                            )
lcc.useCpus = 8
lcc.load_all_lcs()

lcs = lcc.lcs

In [130]:
files2 = os.path.join(f'TESSpklFilesSect{sector}', 'tesslc_*.pkl')
files2 = glob.glob(files2)

In [131]:
#get folded lcs for smaller list
folded = []
for i in range(len(files2)):
    with (open(files2[i], "rb")) as openfile:
        while True:
            try:
                folded.append(pickle.load(openfile))
            except EOFError:
                break

In [132]:
tic = []
avgflux = []
for i in range(len(folded)):
    tic.append(folded[i][0])
    avgflux.append(np.median(folded[i][4]))
    
for i in range(len(lcs)):
    tic.append(nottics2[i])
    avgflux.append(np.median(lcs[i].flux.value))

In [133]:
d = {'TIC': tic, 'avgflux': avgflux}
avg = pd.DataFrame(d)

In [134]:
avg

,TIC,avgflux
0,238188289,3874.810425
1,290132590,658.987889
2,349835414,360.707860
3,300934001,226.355327
4,349788473,222.661370
...,...,...
5250,630986911,188630.783835
5251,278861826,10458.540525
5252,53951580,21058.399692
5253,309853072,14305.984937


In [135]:
highestfluxtics = []
for i in range(len(closestarstics)):
    a = []
    for j in closestarstics[i]:
        a.append(avg[avg['TIC'] == j].avgflux.values[0])
    highestfluxtics.append(closestarstics[i][a.index(max(a))])

In [136]:
highestfluxtics

[269762258,
 263074592,
 278706358,
 261656371,
 152825521,
 139699256,
 139721085,
 139841210,
 214198381,
 144462697,
 389726435,
 160036449,
 229057817,
 204497617,
 441406061,
 152223725,
 25226886,
 280052690,
 358457398,
 141268467,
 33862174,
 267263253,
 79403459,
 79225651,
 394585218,
 99465976,
 201176965,
 389476160,
 261746657,
 262958558,
 290547036,
 263078276,
 33715938,
 303909865,
 181233908,
 271596290,
 325469795,
 220474209,
 278683641,
 201256776,
 92352621,
 92352621,
 441422220,
 89398867,
 147083089,
 206462116,
 117544915,
 139148070,
 139207681,
 91369561,
 44623916,
 144194307,
 126944775,
 126945917,
 167692429,
 410354930,
 32200310,
 290131778,
 281498280,
 220521199,
 234491752,
 38586438,
 55652788,
 349832824,
 147131974,
 91448148,
 147201138,
 121420805,
 89018905,
 126602778,
 141606986,
 197766143,
 266996791,
 403136932,
 219308281,
 206505572,
 260161144,
 403153647,
 265591866,
 219338651,
 231714759,
 419635446,
 290025724,
 54001270,
 44135567

In [137]:
def checkIfDuplicates_2(listOfElems):
    ''' Check if given list contains any duplicates '''    
    setOfElems = set()
    for elem in listOfElems:
        if elem in setOfElems:
            return True
        else:
            setOfElems.add(elem)         
    return False

In [138]:
result = checkIfDuplicates_2(highestfluxtics)
if result:
    print('Yes, list contains duplicates')
else:
    print('No duplicates found in list')   

Yes, list contains duplicates


In [139]:
newlist = [] #eb list with no duplicates, cut down from 3345 to be ones with highest flux values in a radius of 0.1
duplist = []
for i in highestfluxtics:
    if i not in newlist:
        newlist.append(i)
    else:
        duplist.append(i)

In [140]:
with open(f"Sector{sector}/newlistSect{sector}.txt", "w") as output:
    for i in range(len(newlist)):
        output.write(str(newlist[i]))
        output.write("\n")

In [141]:
list1 = []
for element in newlist:
    if element in tics2:
        list1.append(element)

In [142]:
folded2 = []
for i in range(len(list1)):
    with (open(f'TESSpklFilesSect{sector}/tesslc_{list1[i]}folded.pkl', "rb")) as openfile:
        while True:
            try:
                folded2.append(pickle.load(openfile))
            except EOFError:
                break

In [143]:
import matplotlib.pyplot as plt

from matplotlib.backends.backend_pdf import PdfPages

with PdfPages(f"Sector{sector}/finallcsSect{sector}.pdf") as pdf:
    plt.cla()
    for i in range(len(folded2)):
        fig,ax = plt.subplots(2)
        lc = lk.LightCurve(time = folded2[i][3], flux = folded2[i][4], flux_err = folded2[i][5], targetid=folded2[i][0], quality = folded2[i][7])
        lc2.scatter(ax = ax[0])
        lc.remove_outliers(sigma_lower = 7, sigma_upper = 3).scatter(ax = ax[1])
        plt.title(f'TIC{folded2[i][0]}')
        pdf.savefig()
        plt.close();
plt.close()